In [1]:
!git clone https://github.com/jbhayet/semanaia-hackathon/

Cloning into 'semanaia-hackathon'...
remote: Enumerating objects: 188, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 188 (delta 10), reused 13 (delta 4), pack-reused 166 (from 1)
Receiving objects: 100% (188/188), 61.02 MiB | 25.59 MiB/s, done.
Resolving deltas: 100% (98/98), done.
Updating files: 100% (30/30), done.


In [2]:
%cd semanaia-hackathon

/Users/fmayf/Documents/frmx/hackfrancomexicano/semanaia-hackathon


In [7]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [8]:

pip install plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 27.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [plotly]2m1/2 [plotly]
Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
import plotly.express as px

# --- Cargar datos
df_cdmx = pd.read_csv("data/cdmx_data_series.csv")
df_lyon = pd.read_csv("data/lyon_data_series.csv")

# --- Agregar ciudad
df_cdmx["ciudad"] = "CDMX"
df_lyon["ciudad"] = "Lyon"
df_cdmx["station_id"] = df_cdmx["zone_id"]
df_lyon["station_id"] = df_lyon["zone_id"]

# --- Expandir a formato largo
ocup_cols = [f"occupation_{i}" for i in range(24)]
def expandir(df):
    df_long = df.melt(
        id_vars=["station_id", "lon", "lat", "zone_id", "ciudad", "weekday"],
        value_vars=ocup_cols,
        var_name="hour",
        value_name="ocupacion"
    )
    df_long["hour"] = df_long["hour"].str.extract("(\d+)").astype(int)
    return df_long

cdmx_long = expandir(df_cdmx)
lyon_long = expandir(df_lyon)

# --- Asignar fechas por semana (1 fecha por día de la semana)
dias_semana = ['2023-06-05', '2023-06-06', '2023-06-07', '2023-06-08', '2023-06-09', '2023-06-10', '2023-06-11']
cdmx_long["date"] = cdmx_long["weekday"].apply(lambda x: pd.to_datetime(dias_semana[x]))
lyon_long["date"] = lyon_long["weekday"].apply(lambda x: pd.to_datetime(dias_semana[x]))

# --- Timestamp completo
cdmx_long["timestamp"] = cdmx_long["date"] + pd.to_timedelta(cdmx_long["hour"], unit='h')
lyon_long["timestamp"] = lyon_long["date"] + pd.to_timedelta(lyon_long["hour"], unit='h')
cdmx_long["timestamp_str"] = cdmx_long["timestamp"].dt.strftime("%a %H:%M")
lyon_long["timestamp_str"] = lyon_long["timestamp"].dt.strftime("%a %H:%M")

# --- MAPAS 1 y 2: Ocupación por semana (cada frame = día)
def mapa_por_semana(df, ciudad):
    df_dia = df[df["hour"] == 9]  # usar misma hora para comparar días
    fig = px.scatter_mapbox(
        df_dia,
        lat="lat", lon="lon",
        color="ocupacion",
        size="ocupacion",
        animation_frame=df_dia["date"].dt.strftime("%A"),
        hover_name="station_id",
        color_continuous_scale="YlOrRd",
        zoom=11 if ciudad == "CDMX" else 12,
        mapbox_style="carto-positron",
        title=f"🗺️ {ciudad}: Ocupación por Semana (9:00am)"
    )
    fig.update_layout(updatemenus=[{
        "buttons": [{
            "args": [None, {"frame": {"duration": 2000, "redraw": True}, "fromcurrent": True}],
            "label": "▶ Reproducir",
            "method": "animate"
        }],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": True,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }])
    fig.show()

# --- MAPAS 3 y 4: Ocupación por hora durante un día
def mapa_por_dia(df, ciudad, dia_nombre="Friday"):
    dia_idx = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'].index(dia_nombre)
    df_dia = df[df["weekday"] == dia_idx]
    fig = px.scatter_mapbox(
        df_dia,
        lat="lat", lon="lon",
        color="ocupacion",
        size="ocupacion",
        animation_frame="hour",
        hover_name="station_id",
        color_continuous_scale="Turbo",
        zoom=11 if ciudad == "CDMX" else 12,
        mapbox_style="carto-positron",
        title=f"🕓 {ciudad}: Ocupación por Hora – {dia_nombre}"
    )
    fig.update_layout(updatemenus=[{
        "buttons": [{
            "args": [None, {"frame": {"duration": 2000, "redraw": True}, "fromcurrent": True}],
            "label": "▶ Reproducir",
            "method": "animate"
        }],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": True,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }])
    fig.show()

# === EJECUCIÓN DE LOS 4 MAPAS ===
mapa_por_semana(cdmx_long, "CDMX")
mapa_por_semana(lyon_long, "Lyon")
mapa_por_dia(cdmx_long, "CDMX", "Friday")
mapa_por_dia(lyon_long, "Lyon", "Friday")


<>:23: SyntaxWarning: invalid escape sequence '\d'
<>:23: SyntaxWarning: invalid escape sequence '\d'
/var/folders/h9/bjpzl8653tsgcs1c3c7rdzdh0000gn/T/ipykernel_30202/2083885542.py:23: SyntaxWarning: invalid escape sequence '\d'
  df_long["hour"] = df_long["hour"].str.extract("(\d+)").astype(int)
/var/folders/h9/bjpzl8653tsgcs1c3c7rdzdh0000gn/T/ipykernel_30202/2083885542.py:43: DeprecationWarning: *scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/
  fig = px.scatter_mapbox(


ValueError: 
    Invalid element(s) received for the 'size' property of scattermapbox.marker
        Invalid elements include: [-0.211, -0.105, -0.211, -0.105, -0.211, -0.263, -0.158, -0.053, -0.158, -0.474]

    The 'size' property is a number and may be specified as:
      - An int or float in the interval [0, inf]
      - A tuple, list, or one-dimensional numpy array of the above

In [11]:
pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 23.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 29.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 30.2 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: packaging━━━━━━━━━━━━━━━━━━━━━━━━━━━━  8/28 [pillow]f]
    Found existing installation: packaging 25.0━━━━━━━━━━━━━━━  8/28 [pillow]
    Uninstalling packaging-25.0:0m━━━━━━━━━━━━━━━━━━━━━━━━━━━━  8/28 [pillow]
      Successfully uninstalled packaging-25.0━━━━━━━━━━━━━━━━━  8/28 [pillow]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28/28 [streamlit]28 [streamlit]]
Note: you may need to restart the kernel to use updated packages.


In [13]:
import streamlit as st
import pandas as pd
import plotly.express as px

# --- CONFIGURACIÓN ---
st.set_page_config(page_title="Clasificación Estaciones Ecobici CDMX", layout="wide")

# --- CARGA DE DATOS ---
@st.cache_data

def cargar_datos():
    df = pd.read_csv("data/estaciones.csv")
    return df

df = cargar_datos()

# Diccionario de colores y nombres para la clasificación
tipos_dict = {
    1: ("Poco uso", "#ADD8E6"),
    2: ("Mayoría llegadas", "#FFA07A"),
    3: ("Mayoría salidas", "#FFA07A"),
    4: ("Más salidas que llegadas", "#FFD700"),
    5: ("Más llegadas que salidas", "#FFD700"),
    6: ("Equilibrado", "#008000")
}

df["clasificacion_nombre"] = df["tipoestacion"].map(lambda x: tipos_dict[x][0])
df["color"] = df["tipoestacion"].map(lambda x: tipos_dict[x][1])

# --- SIDEBAR ---
st.sidebar.title("🔧 Filtros")
dia_semana = st.sidebar.selectbox("Día de la semana", [
    "Total", "Lunes", "Martes", "Miércoles", "Jueves", "Viernes", "Sábado", "Domingo"
])

# --- SELECCIÓN DE DÍAS ---
mapping_dias = {
    "Lunes": ("luns", "lune"),
    "Martes": ("mars", "mare"),
    "Miércoles": ("mies", "miee"),
    "Jueves": ("jues", "juee"),
    "Viernes": ("vies", "viee"),
    "Sábado": ("sabs", "sabe"),
    "Domingo": ("doms", "dome"),
    "Total": ("totals", "totale")
}
salidas_col, llegadas_col = mapping_dias[dia_semana]

# --- GRÁFICA ---
st.title("📍 Clasificación de Estaciones Ecobici CDMX")
st.markdown(f"### {dia_semana}: Promedios de salidas vs llegadas")

fig = px.scatter_mapbox(
    df,
    lat="lat",
    lon="lon",
    color="clasificacion_nombre",
    color_discrete_map={nombre: color for _, (nombre, color) in tipos_dict.items()},
    size=salidas_col,
    hover_data={
        "idestacion": True,
        salidas_col: True,
        llegadas_col: True,
        "clasificacion_nombre": True
    },
    mapbox_style="carto-positron",
    zoom=11,
    center={"lat": 19.43, "lon": -99.13},
    title=f"CDMX – Clasificación de estaciones por {dia_semana}"
)

st.plotly_chart(fig, use_container_width=True)

# --- TABLA DE DATOS ---
st.markdown("### 📊 Tabla de estaciones")
st.dataframe(df[["idestacion", "lat", "lon", salidas_col, llegadas_col, "clasificacion_nombre"]])


2025-06-06 01:08:21.208 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-06 01:08:21.217 No runtime found, using MemoryCacheStorageManager
2025-06-06 01:08:21.218 No runtime found, using MemoryCacheStorageManager
2025-06-06 01:08:21.219 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-06 01:08:21.220 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-06 01:08:21.220 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-06 01:08:21.225 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-06 01:08:21.226 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


FileNotFoundError: [Errno 2] No such file or directory: 'data/estaciones.csv'